In [6]:
import pandas as pd
#concatonating all the csv files we downloaded from spotify
import glob
path = "/Users/ruthmazurek/Desktop/flatiron_projects/Spotify/spotify-project/Spotify_csv_files/*.csv"
cap = glob.glob(path)

In [2]:
#get in the file with all the csv files
cd Spotify_csv_files

/Users/ruthmazurek/Desktop/flatiron_projects/Spotify/spotify-project/Spotify_csv_files


In [3]:
charts = [i.replace('/Users/ruthmazurek/Desktop/flatiron_projects/Spotify/spotify-project/Spotify_csv_files/',"") for i in cap]


In [4]:
def get_ids(csv_name):
    track_ids = []
    chart = pd.read_csv(csv_name)
    for i in range(1,len(chart)):
        track = chart['Unnamed: 4'][i].split("/")[4]
        track_ids.append(track)
    return track_ids

In [7]:
all_ids = []
for thing in charts:   
    all_ids.extend(get_ids(thing))

In [8]:
def get_artists(csv_name):
    artist_spotify = []
    chart = pd.read_csv(csv_name)
    for i in range(1,len(chart)):
        artist = chart['Unnamed: 2'][i]
        artist_spotify.append(artist)
    return artist_spotify

In [9]:
all_artists = []
for thing in charts:   
    all_artists.extend(get_artists(thing))

In [10]:
def get_names(csv_name):
    names = []
    chart = pd.read_csv(csv_name)
    for i in range(1,len(chart)):
        artist = chart['Unnamed: 1'][i]
        names.append(artist)
    return names

In [11]:
all_names = []
for thing in charts:   
    all_names.extend(get_names(thing))

In [12]:
def get_positions(csv_name):
    positions = []
    chart = pd.read_csv(csv_name)
    for i in range(1,len(chart)):
        position = chart['Unnamed: 0'][i]
        positions.append(position)
    return positions

In [13]:
all_positions = []
for thing in charts:   
    all_positions.extend(get_positions(thing))

In [14]:
def get_streams(csv_name):
    streams = []
    chart = pd.read_csv(csv_name)
    for i in range(1,len(chart)):
        stream = chart['Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.'][i]
        streams.append(stream)
    return streams


In [15]:
all_streams = []
for thing in charts:   
    all_streams.extend(get_streams(thing))

In [16]:
week_numbers = []
for i in range(0,155):
    counter=200
    while counter != 0:
        week_numbers.append(i)
        counter = counter-1

In [17]:
#putting all the data in 1 tuple
all_data = []
for thing in range(0,len(all_names)):
    new_tuple = (all_ids[thing], week_numbers[thing], int(all_positions[thing]), all_names[thing],
                 all_artists[thing],int(all_streams[thing]))
    all_data.append(new_tuple)

In [ ]:
#dropping the nans from the data sets - there were around 4 out of 31,000 that needed to be dropped
all_data_pandas=pd.DataFrame(all_data)
all_data_pandas_clean=all_data_pandas.dropna()

In [ ]:
#naming the columns
all_data_pandas_clean.columns['id','week','rank','song','artist','strems']

In [ ]:
#creating a new tuple with the dropped Nans
all_data_dropna=tuple(all_data_pandas_clean.itertuples(index=False, name=None))

In [ ]:
#inserting the tuple into the database
import mysql.connector

In [ ]:
import config_rob_database
cnx = mysql.connector.connect(
    host = config_rob_database.host,
    user = config_rob_database.user,
    passwd = config_rob_database.passwd)

In [ ]:
cursor = cnx.cursor()
cnx.database = 'spotify'

In [ ]:
create_table_spotify = """
CREATE TABLE spotify_weekly
(track_id VARCHAR(225),
week_of VARCHAR(225),
chart_position INT,
track_name VARCHAR(225),
artist VARCHAR(225),
num_of_streams INT);
"""

In [ ]:
cursor.execute(create_table_spotify)

In [ ]:
insert_stm = """INSERT INTO spotify_weekly (track_id, week_of, chart_position, track_name, artist,num_of_streams) 
    VALUES (%s, %s, %s, %s, %s, %s)"""
cursor.executemany(insert_stm, all_data_dropna)
cnx.commit()

In [ ]:
#now using spotifys API and spotify to get data from spotify

In [ ]:
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
#general way to grab audio analysis using the song ID
credentials = spotipy.oauth2.SpotifyClientCredentials(client_id='d1752e10625849959f530b56af4458e0', client_secret='057f9a3c8e584dbd9b6c27cf2a224963')
spotify = spotipy.Spotify(client_credentials_manager=credentials)
#'7qiZfU4dY1lWllzX7mPBI3' is a song ID
print (spotify.audio_analysis('7qiZfU4dY1lWllzX7mPBI3'))

In [ ]:
#general way to grab audio features using the song ID
credentials = spotipy.oauth2.SpotifyClientCredentials(client_id='d1752e10625849959f530b56af4458e0', client_secret='057f9a3c8e584dbd9b6c27cf2a224963')
spotify = spotipy.Spotify(client_credentials_manager=credentials)
#'7qiZfU4dY1lWllzX7mPBI3' is a song ID
print (spotify.audio_features(['7qiZfU4dY1lWllzX7mPBI3']))

In [ ]:
#uses the list I made of all_ids - all the spotify song ids I want - and grabs all the audio features
all_audio_features = []
for thing in all_ids:
    all_audio_features.append((spotify.audio_features([f'{thing}'])))

In [ ]:
#uses the list I made of artists - uses the artists names - and grabs all the audio features
artist_data = []
for thing in all_artists:
    artist_data.append((spotify.artists([f'{thing}'])))